# Deep Connectivity Playground

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [102]:
# calculate the cosine distance between all pairs of time series

import numpy as np
from scipy import spatial, linalg, stats

n_timepoints = 652
n_regions = 6
timeseries = np.random.rand(n_regions, n_timepoints)

dist = spatial.distance.pdist(timeseries, 'correlation')

# NOTE np.isclose(1 - dist, pearson_corr[np.triu_indices(n_regions, k=1)])

pearson_corr = np.corrcoef(timeseries)
spearman_corr = stats.spearmanr(timeseries, axis=1).correlation
pcorr = np.zeros_like(spearman_corr)

corr_inv = linalg.inv(spearman_corr)
for i in range(corr_inv.shape[0]):
    for j in range(corr_inv.shape[1]):
        if i != j:  # off-diagonal
            pcorr[i, j] = -corr_inv[i, j] / np.sqrt(corr_inv[i, i] * corr_inv[j, j])

pcorr[np.diag_indices_from(pcorr)] = 1.0  # set diagonal to 1

def distance_matrix(data):
    """Compute the pairwise distance matrix of a dataset."""
    dist_matrix = np.sqrt(np.sum((data[:, np.newaxis, :] - data[np.newaxis, :, :]) ** 2, axis=-1))
    return dist_matrix

def double_centered(dist_matrix):
    """Apply double centering to a distance matrix."""
    row_means = np.mean(dist_matrix, axis=1, keepdims=True)
    col_means = np.mean(dist_matrix, axis=0, keepdims=True)
    grand_mean = np.mean(dist_matrix)
    centered_matrix = dist_matrix - row_means - col_means + grand_mean
    return centered_matrix

def distance_covariance(X, Y):
    """Calculate distance covariance between two datasets."""
    dist_X = distance_matrix(X)
    dist_Y = distance_matrix(Y)
    A = double_centered(dist_X)
    B = double_centered(dist_Y)
    n = X.shape[0]
    dCovXY = np.sqrt(np.sum(A * B) / (n * n))
    return dCovXY

def distance_correlation(X, Y):
    """Compute the distance correlation between two datasets."""
    dCovXY = distance_covariance(X, Y)
    dVarX = distance_covariance(X, X)
    dVarY = distance_covariance(Y, Y)
    dCorXY = dCovXY / np.sqrt(dVarX * dVarY)
    return dCorXY

dcor = np.zeros((n_regions, n_regions))

for i in range(n_regions):
    for j in range(i, n_regions):  # No need to compute for j < i, matrix is symmetric
        if i == j:
            dcor[i, j] = 1.0
        else:
            _dcor = distance_correlation(timeseries[i, :].reshape(-1, 1),
                                         timeseries[j, :].reshape(-1, 1))
            dcor[i, j] = dcor[j, i] = _dcor
dcor

# NOTE decision is to use spearman/partial for regions,
#      spearman/partial/distance-corr for networks

array([[1.        , 0.06693988, 0.05688217, 0.04820034, 0.04553769,
        0.09122409],
       [0.06693988, 1.        , 0.05509206, 0.05243047, 0.04981787,
        0.04698553],
       [0.05688217, 0.05509206, 1.        , 0.08088801, 0.07674842,
        0.05699306],
       [0.04820034, 0.05243047, 0.08088801, 1.        , 0.05118885,
        0.0507496 ],
       [0.04553769, 0.04981787, 0.07674842, 0.05118885, 1.        ,
        0.06942166],
       [0.09122409, 0.04698553, 0.05699306, 0.0507496 , 0.06942166,
        1.        ]])